In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth' ,999)

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.metrics import plot_confusion_matrix, classification_report, f1_score, mean_squared_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(3792)

In [4]:
kickoff_df = pd.read_csv('../data/kickoff_no_nulls.csv')
kickoff_df.drop(columns ='Unnamed: 0', inplace=True)
kickoff_df.head(1)

,gameId,playId,playDescription,quarter,possessionTeam,kickerId,returnerId,yardlineSide,yardlineNumber,gameClock,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,specialTeamsSafeties
0,2018090600,677,"M.Bosher kicks 64 yards from ATL 35 to PHI 1. S.Gibson to PHI 31 for 30 yards (D.Kazee, D.Riley).",1,ATL,37267.0,44979,ATL,35,05:01:00,NP,NP,0,0,3,64.0,30.0,75,4.06,D,R,R,C,C,ATL 83,ATL 22,ATL 27,8-0-2,ATL 17; ATL 22


In [7]:
numerics = ['uint8', 'float64', 'int64', 'int32', 'float16', 'float32']

kickoff_numerics = kickoff_df.select_dtypes(include=numerics)
kickoff_numerics.head()

,gameId,playId,quarter,kickerId,yardlineNumber,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime
0,2018090600,677,1,37267.0,35,0,0,3,64.0,30.0,75,4.06
1,2018090600,1606,2,37267.0,35,5,3,6,68.0,13.0,45,3.63
2,2018090900,36,1,33469.0,35,0,0,0,69.0,24.0,75,3.83
3,2018090900,400,1,39470.0,35,0,7,0,61.0,28.0,45,4.21
4,2018090900,1992,2,39470.0,35,0,20,0,65.0,25.0,75,3.99


In [13]:
X = kickoff_numerics.drop(columns='kickReturnYardage')
y = kickoff_numerics['kickReturnYardage']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=3792,
                                                   train_size=.75)

In [14]:
lr = LinearRegression()

In [15]:
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
lr.score(X_train, y_train) , lr.score(X_test, y_test)

(0.16735258276269638, 0.13094927683547275)

In [22]:
lr.coef_[2]

1.6204470736997851

In [20]:
pd.DataFrame(zip( X.columns, lr.coef_), 
             columns=['Feature', 'Coef'])

,Feature,Coef
0,gameId,-1.766379e-07
1,playId,-1.036489e-03
2,quarter,1.620447e+00
3,kickerId,-1.564126e-04
4,yardlineNumber,1.356343e-01
5,penaltyYards,-2.819545e-01
6,preSnapHomeScore,-4.391865e-02
7,preSnapVisitorScore,-4.008917e-02
8,kickLength,7.159659e-01
9,absoluteYardlineNumber,-1.022048e-02


In [26]:
round((mean_squared_error(y_true=y_test, y_pred=lr.predict(X_test)) ** .5), 4)

9.1482